add aqui danilove

In [12]:
import pandas as pd
df_final = pd.read_csv("../dados/df_preparado.csv")

In [13]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from typing import List, Dict, Any
import json 
import re

df = df_final.head(200).copy()

#print(df['setor_economico'].value_counts())

# Carrega variáveis do .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [14]:
def criar_agrupamentos_setores(setores_unicos: List[str], num_grupos: int = 5) -> Dict[str, List[str]]:
    """
    Cria agrupamentos inteligentes de setores econômicos usando Gemini
    """
    prompt = f"""
    Analise os seguintes setores econômicos e crie {num_grupos} agrupamentos lógicos baseados em similaridade de atividade, cadeia produtiva ou segmento de mercado:

    Setores: {', '.join(setores_unicos)}

    Instruções:
    1. Crie {num_grupos} grupos distintos
    2. Cada grupo deve ter um nome descritivo com até 4 palavras
    3. Agrupe setores relacionados por atividade econômica similar
    4. Retorne no formato JSON estruturado

    Formato de resposta (JSON):
    {{
        "grupo_1": {{
            "nome": "Nome do Grupo",
            "setores": ["setor1", "setor2"]
        }},
        "grupo_2": {{
            "nome": "Nome do Grupo",
            "setores": ["setor3", "setor4"]
        }}
    }}
    """
    try:
        response = model.generate_content(prompt)
        response_text = response.text.strip()

        # Limpa o início e o fim se vierem marcadores de bloco de código
        if response_text.startswith("```json"):
            response_text = response_text[len("```json"):].strip()
        elif response_text.startswith("```"):
            response_text = response_text[len("```"):].strip()
        if response_text.endswith("```"):
            response_text = response_text[:-3].strip()

        # Remove vírgulas antes de fechar colchetes ou chaves
        response_text = re.sub(r",\s*([}$$])", r"\1", response_text)
            
        agrupamentos = json.loads(response_text)
        return agrupamentos
    except Exception as e:
        print(f"Erro ao criar agrupamentos: {e}")
        return {}


In [15]:
setores_unicos = df['setor_economico'].dropna().unique().tolist()

#agrupamentos = criar_agrupamentos_setores(setores_unicos, 15)

In [16]:
print(df['setor_economico'].dropna().unique().tolist())

['Fabricação de Peças e Acessórios para o Sistema Motor de Veículos Automotores', 'Gestão e Administração da Propriedade Imobiliária', 'Comércio Atacadista de Combustíveis Sólidos, Líquidos e Gasosos, Exceto Gás Natural e Glp', 'Construção de Edifícios# Construção de Rodovias e Ferrovias', 'Comércio de Peças e Acessórios para Veículos Automotores', 'Fabricação de Aparelhos Eletromédicos e Eletroterapêuticos e Equipamentos de Irradiação', 'Comércio Varejista de Gás Liqüefeito de Petróleo (Glp)', 'Comércio Varejista de Combustíveis para Veículos Automotores', 'Fabricação de Aparelhos e Equipamentos de Medida, Teste e Controle', 'Serviços de Engenharia', 'Fabricação de Resinas Termofixas# Fabricação de Resinas Termoplásticas', 'Fabricação de Embalagens de Material Plástico', 'Aluguel de Máquinas e Equipamentos para Construção Sem Operador', 'Fabricação de Produtos Derivados do Petróleo, exceto Produtos do Refino', 'Fabricação de Medicamentos para Uso Humano# Comércio Atacadista de Produto

In [17]:
def rotular_setor(setor: str, agrupamentos: Dict[str, Any]) -> str:
    """
    Rotula um setor específico baseado nos agrupamentos criados
    """
    for grupo_key, grupo_info in agrupamentos.items():
        if setor in grupo_info.get('setores', []):
            return grupo_info.get('nome', grupo_key)
    
    # Se não encontrou correspondência exata, usa o Gemini para classificar
    prompt = f"""
    Baseado nos seguintes grupos existentes, classifique o setor "{setor}":

    Grupos disponíveis:
    {json.dumps({k: v['nome'] for k, v in agrupamentos.items()}, indent=2, ensure_ascii=False)}

    Retorne apenas o nome do grupo mais apropriado para o setor "{setor}".
    Regras:
    1. Se não houver correspondência clara, responda "Outros"
    2. Não crie novos grupos
    3. Use exatamente os nomes dos grupos fornecidos
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Erro ao rotular setor {setor}: {e}")
        return "Não Classificado"

def classificar_dataframe_setores(df: pd.DataFrame, coluna_setor: str, num_grupos: int = 5) -> pd.DataFrame:
    """
    Função principal que classifica todo o dataframe
    """
    # Obtém setores únicos
    setores_unicos = df[coluna_setor].dropna().unique().tolist()
    
    print(f"Encontrados {len(setores_unicos)} setores únicos")
    print("Criando agrupamentos com Gemini...")
    
    # Cria agrupamentos
    agrupamentos = criar_agrupamentos_setores(setores_unicos, num_grupos)
    
    if not agrupamentos:
        print("Falha ao criar agrupamentos")
        return df
    
    print("Agrupamentos criados:")
    for grupo_key, grupo_info in agrupamentos.items():
        print(f"- {grupo_info.get('nome', grupo_key)}: {len(grupo_info.get('setores', []))} setores")
    
    # Aplica rótulos
    print("Aplicando rótulos...")
    df['grupo_setor'] = df[coluna_setor].apply(lambda x: rotular_setor(x, agrupamentos) if pd.notna(x) else "Não Informado")
    
    return df

In [18]:
classificar_dataframe_setores(df, 'setor_economico', 8)

Encontrados 42 setores únicos
Criando agrupamentos com Gemini...
Agrupamentos criados:
- Indústria Automobilística: 4 setores
- Construção e Engenharia: 8 setores
- Comércio de Combustíveis: 4 setores
- Indústria Química e Plástica: 6 setores
- Saúde e Farmácia: 5 setores
- Indústria Elétrica e Eletrônica: 8 setores
- Comércio Varejista e Serviços: 8 setores
- Logística e Imobiliário: 7 setores
Aplicando rótulos...


,id,id_unique,numero_sei,ano_documento,assinaturas,descricao_tipo_documento,descricao_tipo_processo,numero_processo,id_unidade,sigla_unidade,...,mercado_relevante,documentos_relacionados,descricao_especificacao,data_processo,_version_,corpo_texto,decisao_tribunal,conteudo,diferenca_dias,grupo_setor
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,jurisprudencia__1390607,1390607,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.004558/2019-05,110000967.0,GAB1,...,"pistões de motor, bronzinas, camisas, pinos, b...","{id_unidade:110000967,protocolo_documento:1085...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",2019-09-13,1800936057604145200,NaN,vazio,SEI/CADE - 1390607 - Voto Processo Administrat...,1714,Indústria Automobilística
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,jurisprudencia__1384594,1384594,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.004974/2015-71,110000969.0,GAB3,...,Mercado imobiliário,"{id_unidade:110000969,protocolo_documento:0257...",Influencia de conduta uniforme - COFECI,2015-05-21,1800936002855895000,NaN,condenacao,SEI/CADE - 1384594 - Voto Processo Administrat...,3276,Logística e Imobiliário
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,jurisprudencia__1384685,1384685,2024,Outras,Voto,Processo Administrativo,08700.005915/2022-40,110001021.0,DIAP,...,Postos de gasolina,"{id_unidade:110001021,protocolo_documento:1100...",Ministério Público do Estado de Minas Gerais,2022-08-08,1800936088917770200,voto\r\n\r\nI. BREVE RESUMO DO PROCESSO&nbsp;\...,arquivamento,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,640,Comércio de Combustíveis
3,c8499708-5778-45d9-8914-9ab97b77c1fd,jurisprudencia__1377990,1377990,2024,Alexandre Cordeiro Macedo,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000955.0,GAB-PRES,...,obras de engenharia civil# Construção de infra...,"{id_unidade:110000955,protocolo_documento:0273...",PAC Favelas - AL 09/2016 - Favelas,2016-11-22,1800936018528960500,NaN,condenacao,SEI/CADE - 1377990 - Voto Processo Administrat...,2709,Construção e Engenharia
4,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,jurisprudencia__1377023,1377023,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000969.0,GAB3,...,obras de engenharia civil# Construção de infra...,"{id_unidade:110000969,protocolo_documento:0273...",PAC Favelas - AL 09/2016 - Favelas,2016-11-22,1800936018619138000,NaN,condenacao,SEI/CADE - 1377023 - Voto Processo Administrat...,2709,Construção e Engenharia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,fe719fd6-f685-4e89-ad9f-943fcff4f602,jurisprudencia__0919607,919607,2021,Paula Farani de Azevedo Silveira,Voto Processo Administrativo,Processo Administrativo,08700.002407/2017-42,110000970.0,GAB4,...,"Linha de Assinante Digital Assimétrica - ADSL,...","{id_unidade:110000970,protocolo_documento:0368...",NaN,2017-04-19,1800936007921565700,NaN,arquivamento,SEI/CADE - 0919607 - Voto Processo Administrat...,1524,Indústria Elétrica e Eletrônica
196,2acedd15-07da-4643-a856-5d791e2bd40c,jurisprudencia__0919624,919624,2021,Paula Farani de Azevedo Silveira,Voto Processo Administrativo,Processo Administrativo,08700.008751/2015-83,110000970.0,GAB4,...,Mercado de transporte marítimo de contêineres#...,"{id_unidade:110000970,protocolo_documento:0120...",OGMO BVC,2015-08-31,1800936004634280000,NaN,condenacao,SEI/CADE - 0919624 - Voto Processo Administrat...,2121,Logística e Imobiliário
197,e8ebd86a-9a51-443f-9236-edd08512196c,jurisprudencia__0920855,920855,2021,Sérgio Costa Ravagnani,Voto Processo Administrativo,Processo Administrativo,08700.002407/2017-42,110000971.0,GAB5,...,"Linha de Assinante Digital Assimétrica - ADSL,...","{id_unidade:110000971,protocolo_documento:0368...",NaN,2017-04-19,1800936021327609900,NaN,arquivamento,SEI/CADE - 0920855 - Voto Processo Administrat...,1524,Indústria Elétrica e Eletrônica
198,b9945844-7c20-4

In [19]:
df['grupo_setor'].value_counts()

grupo_setor
Construção e Engenharia            40
Comércio de Combustíveis           30
Comércio Varejista e Serviços      30
Indústria Elétrica e Eletrônica    29
Logística e Imobiliário            28
Indústria Automobilística          21
Indústria Química e Plástica       18
Saúde e Farmácia                    4
Name: count, dtype: int64

In [20]:
col = 'grupo_setor'
cols = list(df.columns)
cols.insert(6, cols.pop(cols.index(col)))
df = df[cols]

df.head(3)

,id,id_unique,numero_sei,ano_documento,assinaturas,descricao_tipo_documento,grupo_setor,descricao_tipo_processo,numero_processo,id_unidade,...,link_documento,mercado_relevante,documentos_relacionados,descricao_especificacao,data_processo,_version_,corpo_texto,decisao_tribunal,conteudo,diferenca_dias
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,jurisprudencia__1390607,1390607,2024,Outras,Voto Processo Administrativo,Indústria Automobilística,Processo Administrativo,08700.004558/2019-05,110000967.0,...,http://sei.cade.gov.br/sei//modulos/pesquisa/m...,"pistões de motor, bronzinas, camisas, pinos, b...","{id_unidade:110000967,protocolo_documento:1085...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",2019-09-13,1800936057604145200,NaN,vazio,SEI/CADE - 1390607 - Voto Processo Administrat...,1714
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,jurisprudencia__1384594,1384594,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Logística e Imobiliário,Processo Administrativo,08700.004974/2015-71,110000969.0,...,http://sei.cade.gov.br/sei//modulos/pesquisa/m...,Mercado imobiliário,"{id_unidade:110000969,protocolo_documento:0257...",Influencia de conduta uniforme - COFECI,2015-05-21,1800936002855895000,NaN,condenacao,SEI/CADE - 1384594 - Voto Processo Administrat...,3276
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,jurisprudencia__1384685,1384685,2024,Outras,Voto,Comércio de Combustíveis,Processo Administrativo,08700.005915/2022-40,110001021.0,...,http://sei.cade.gov.br/sei//modulos/pesquisa/m...,Postos de gasolina,"{id_unidade:110001021,protocolo_documento:1100...",Ministério Público do Estado de Minas Gerais,2022-08-08,1800936088917770200,voto\r\n\r\nI. BREVE RESUMO DO PROCESSO&nbsp;\...,arquivamento,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,640


In [21]:
df['grupo_setor'].value_counts()

grupo_setor
Construção e Engenharia            40
Comércio de Combustíveis           30
Comércio Varejista e Serviços      30
Indústria Elétrica e Eletrônica    29
Logística e Imobiliário            28
Indústria Automobilística          21
Indústria Química e Plástica       18
Saúde e Farmácia                    4
Name: count, dtype: int64

In [22]:
df.to_csv("../dados/df_setorizado.csv", index=False)